**Marketing Campaign Dataset**

In [558]:
#Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import KFold 
from sklearn.model_selection import RepeatedKFold

In [559]:
#Load data
marketing_train = pd.read_csv("marketing_tr.csv")

**Checking the shape **

In [560]:
marketing_train.shape

(7414, 21)

**Checking the Head**

In [561]:
marketing_train.head(5)

,custAge,profession,marital,schooling,housing,loan,contact,month,day_of_week,campaign,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,pmonths,pastEmail,responded
0,55.0,admin.,single,university.degree,no,no,cellular,nov,mon,1,...,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,999.0,0,no
1,NaN,blue-collar,married,NaN,no,no,cellular,jul,mon,1,...,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,999.0,0,no
2,42.0,technician,married,high.school,no,no,telephone,may,mon,1,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,999.0,0,no
3,55.0,management,divorced,NaN,yes,yes,cellular,jul,wed,2,...,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,999.0,0,no
4,NaN,admin.,divorced,university.degree,yes,no,cellular,may,tue,5,...,1,failure,-1.8,92.893,-46.2,1.291,5099.1,999.0,1,no


**Exploratory Data Analysis**

In [562]:
marketing_train['schooling'] = marketing_train['schooling'].replace("illiterate", "unknown")
marketing_train['schooling'] = marketing_train['schooling'].replace(["basic.4y","basic.6y","basic.9y","high.school","professional.course"], "high.school")

In [563]:
marketing_train['month'] = marketing_train['month'].replace(["oct","nov","dec"], "dec")
marketing_train['month'] = marketing_train['month'].replace(["aug","jul","sep"], "sep")
marketing_train['month'] = marketing_train['month'].replace(["apr","jun","may","mar"], "jun")

**Replacing Yes & No with 1's & 0's **

In [564]:
marketing_train['marital'] = marketing_train['marital'].replace("unknown", "married")
marketing_train['loan'] = marketing_train['loan'].replace("unknown", "no")
marketing_train['housing'] = marketing_train['housing'].replace("unknown", "no")
marketing_train['housing'] = marketing_train['housing'].replace("yes",1)
marketing_train['housing'] = marketing_train['housing'].replace("no", 0)
marketing_train['loan'] = marketing_train['loan'].replace("yes", 1)
marketing_train['loan'] = marketing_train['loan'].replace("no", 0)

In [565]:
marketing_train['profession'] = marketing_train['profession'].replace(["management","unknown","unemployed","admin."], "admin.")
marketing_train['profession'] = marketing_train['profession'].replace(["blue-collar","housemaid","services","self-employed","entrepreneur","technician"], "blue-collar")

**Missing Value Analysis**

In [567]:
#Create dataframe with missing percentage
missing_val = pd.DataFrame(marketing_train.isnull().sum())

#Reset index
missing_val = missing_val.reset_index()

#Rename variable
missing_val = missing_val.rename(columns = {'index': 'Variables', 0: 'Missing_percentage'})

#Calculate percentage
missing_val['Missing_percentage'] = (missing_val['Missing_percentage']/len(marketing_train))*100

#descending order
missing_val = missing_val.sort_values('Missing_percentage', ascending = False).reset_index(drop = True)

In [568]:
missing_val.head()

,Variables,Missing_percentage
0,schooling,29.066631
1,custAge,24.332344
2,day_of_week,9.589965
3,poutcome,0.000000
4,pastEmail,0.000000


In [569]:
marketing_train['schooling'].value_counts()

high.school          3473
university.degree    1554
unknown               232
Name: schooling, dtype: int64

**Imputing Missing Values**

In [570]:
#imputation method
#Actual value = 29
#Mean = 40.01
#Median = 38

In [571]:
#Checking value
marketing_train['custAge'].loc[70]

29.0

In [572]:
#create missing value
marketing_train['custAge'].loc[70] = np.nan

C:\Users\ssn\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [573]:
marketing_train['custAge'].loc[70]

nan

In [574]:
#Impute with median - CustAge.
marketing_train['custAge'] = marketing_train['custAge'].fillna(marketing_train['custAge'].median())

In [575]:
marketing_train['custAge'].loc[70]

38.0

**#Imputing Schooling**

In [576]:
marketing_train['schooling'] = marketing_train.groupby("profession")['schooling'].transform(lambda x: x.fillna(x.mode()[0]))

** #Imputing "day_of_week" **

In [577]:
marketing_train['day_of_week'] = marketing_train.groupby(["profession",'month'])['day_of_week'].transform(lambda x: x.fillna(x.mode()[0]))

In [578]:
missing_val = pd.DataFrame(marketing_train.isnull().sum())

In [579]:
missing_val

,0
custAge,0
profession,0
marital,0
schooling,0
housing,0
loan,0
contact,0
month,0
day_of_week,0
campaign,0


In [580]:
#Converting Categorical to Numerical
marketing_train=pd.get_dummies(marketing_train,columns=['profession',
            'marital','schooling','contact','month','day_of_week','poutcome'])

In [581]:
marketing_train.shape

(7414, 37)

In [582]:
marketing_train.head()

,custAge,housing,loan,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,month_jun,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,55.0,0,0,1,999,0,-0.1,93.200,-42.0,4.191,...,0,0,0,1,0,0,0,0,1,0
1,38.0,0,0,1,999,0,1.4,93.918,-42.7,4.960,...,0,1,0,1,0,0,0,0,1,0
2,42.0,0,0,1,999,0,1.1,93.994,-36.4,4.857,...,1,0,0,1,0,0,0,0,1,0
3,55.0,1,1,2,999,0,1.4,93.918,-42.7,4.962,...,0,1,0,0,0,0,1,0,1,0
4,38.0,1,0,5,999,1,-1.8,92.893,-46.2,1.291,...,1,0,0,0,0,1,0,1,0,0


In [583]:
def vif_cal(input_data, dependent_col):
    import statsmodels.formula.api as sm
    x_vars = input_data.drop([dependent_col],axis=1)
    xvar_names = x_vars.columns
    for i in range(0,len(xvar_names)):
        y = x_vars[xvar_names[i]]
        x = x_vars[xvar_names.drop(xvar_names[i])]
        rsq = sm.ols("y~x",x_vars).fit().rsquared
        vif = round(1/(1-rsq),2)
        print(xvar_names[i], "VIF: ", vif)
        
vif_cal(marketing_train,'responded')

custAge VIF:  1.41
housing VIF:  1.02
loan VIF:  1.01
campaign VIF:  1.05
pdays VIF:  88765.85
previous VIF:  7.54
emp.var.rate VIF:  61.77
cons.price.idx VIF:  9.61
cons.conf.idx VIF:  3.83
euribor3m VIF:  129.37
nr.employed VIF:  42.81
pmonths VIF:  88068.52
pastEmail VIF:  2.8
profession_admin. VIF:  inf


C:\Users\ssn\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':


profession_blue-collar VIF:  inf
profession_retired VIF:  inf
profession_student VIF:  inf
marital_divorced VIF:  inf
marital_married VIF:  inf
marital_single VIF:  inf
schooling_high.school VIF:  inf
schooling_university.degree VIF:  inf
schooling_unknown VIF:  inf
contact_cellular VIF:  inf
contact_telephone VIF:  inf
month_dec VIF:  inf
month_jun VIF:  inf
month_sep VIF:  inf
day_of_week_fri VIF:  inf
day_of_week_mon VIF:  inf
day_of_week_thu VIF:  inf
day_of_week_tue VIF:  inf
day_of_week_wed VIF:  inf
poutcome_failure VIF:  inf
poutcome_nonexistent VIF:  inf
poutcome_success VIF:  inf


In [584]:
# Avoiding the Dummy Variable Trap
marketing_train = marketing_train.drop(['profession_blue-collar'],1)
marketing_train = marketing_train.drop(['poutcome_success'],1)
marketing_train = marketing_train.drop(['month_jun'],1)
marketing_train = marketing_train.drop(['poutcome_nonexistent'],1)
marketing_train = marketing_train.drop(['marital_divorced'],1)
marketing_train = marketing_train.drop(['contact_cellular'],1)
marketing_train = marketing_train.drop(['pmonths'],1)
marketing_train = marketing_train.drop(['euribor3m'],1)
marketing_train = marketing_train.drop(['nr.employed'],1)
marketing_train = marketing_train.drop(['day_of_week_thu'],1)
marketing_train = marketing_train.drop(['schooling_unknown'],1)
marketing_train = marketing_train.drop(['schooling_high.school'],1)
marketing_train = marketing_train.drop(['previous'],1)

In [585]:
vif_cal(marketing_train,'responded')

custAge VIF:  1.4
housing VIF:  1.01
loan VIF:  1.01
campaign VIF:  1.04
pdays VIF:  1.81
emp.var.rate VIF:  4.39
cons.price.idx VIF:  3.88
cons.conf.idx VIF:  1.58
pastEmail VIF:  2.19
profession_admin. VIF:  1.57
profession_retired VIF:  1.25
profession_student VIF:  1.1
marital_married VIF:  2.54
marital_single VIF:  2.8
schooling_university.degree VIF:  1.57
contact_telephone VIF:  3.13
month_dec VIF:  1.36
month_sep VIF:  2.66
day_of_week_fri VIF:  1.54
day_of_week_mon VIF:  1.72
day_of_week_tue VIF:  1.57
day_of_week_wed VIF:  1.54
poutcome_failure VIF:  1.83


In [586]:
X = marketing_train.drop('responded',axis=1)
X=np.array(X)
y = marketing_train['responded']

**Imbalanced Dataset**,
****K-Fold Cross Validation****

**10-Fold cross validation**

In [587]:
kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=None) 

In [588]:
for train_index, test_index in  kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

**Applying StandardScalar**

In [589]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_train_scale = scale.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scale)
X_test_scale =scale.fit_transform(X_test)
X_test = pd.DataFrame(X_test_scale)

In [590]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1.679830,-1.041783,-0.427477,-0.567379,0.199327,-0.096017,-0.637395,-0.311475,-0.290227,1.343049,...,1.581471,1.481851,-0.757755,2.655081,-0.699649,-0.459664,1.670771,-0.472527,-0.460381,-0.345877
1,-0.164124,-1.041783,-0.427477,-0.567379,0.199327,0.860566,0.604635,-0.461917,-0.290227,-0.744574,...,-0.632323,-0.674832,-0.757755,-0.376636,1.429288,-0.459664,1.670771,-0.472527,-0.460381,-0.345877
2,0.269747,-1.041783,-0.427477,-0.567379,0.199327,0.669249,0.736104,0.892065,-0.290227,-0.744574,...,-0.632323,-0.674832,1.319688,-0.376636,-0.699649,-0.459664,1.670771,-0.472527,-0.460381,-0.345877
3,1.679830,0.959892,2.339307,-0.194730,0.199327,0.860566,0.604635,-0.461917,-0.290227,1.343049,...,-0.632323,1.481851,-0.757755,-0.376636,1.429288,-0.459664,-0.598526,-0.472527,2.172114,-0.345877
4,-0.164124,0.959892,-0.427477,0.923220,0.199327,-1.180143,-1.168458,-1.214130,0.520781,1.343049,...,-0.632323,1.481851,-0.757755,-0.376636,-0.699649,-0.459664,-0.598526,2.116283,-0.460381,2.891202


In [316]:
#Applying PCA
#from sklearn.decomposition import PCA
#pca=PCA()
#X_train=pca.fit_transform(X_train)

In [317]:
#np.cumsum(pca.explained_variance_ratio_)

array([0.12752921, 0.22169254, 0.29476343, 0.35895667, 0.42120141,
       0.47928334, 0.53226119, 0.58411533, 0.6344186 , 0.68088729,
       0.72610798, 0.76738678, 0.80734317, 0.84455354, 0.88133735,
       0.91454443, 0.93857445, 0.95594253, 0.96707922, 0.97804645,
       0.98733671, 0.99443926, 1.        ])

**Building the Model**

**Decision Tree Model**

In [591]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

**Prediction**

In [592]:
predictions = dtree.predict(X_test)
print(classification_report(y_test,predictions))     

             precision    recall  f1-score   support

         no       0.90      0.90      0.90       649
        yes       0.31      0.30      0.31        92

avg / total       0.83      0.83      0.83       741



In [593]:
print(confusion_matrix(y_test,predictions))
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

[[586  63]
 [ 64  28]]


0.8286099865047234

In [594]:
#n-estimator =900
#F-score of "yes"-0.31
#n-estimator =600
#F-score of "yes"-0.35

**Random Forest model**

In [595]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=600)

In [596]:
#Fit the model
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [597]:
predictions = rfc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

         no       0.90      0.97      0.93       649
        yes       0.55      0.26      0.35        92

avg / total       0.86      0.88      0.86       741



In [598]:
print(confusion_matrix(y_test,predictions))
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

[[629  20]
 [ 68  24]]


0.8812415654520918

**Logistic regression model**

In [599]:
#Fit the model
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [600]:
#prediction
predictions = logmodel.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

         no       0.90      0.98      0.94       649
        yes       0.70      0.25      0.37        92

avg / total       0.88      0.89      0.87       741



In [601]:
accuracy_score(y_test, predictions)
print(confusion_matrix(y_test,predictions))

[[639  10]
 [ 69  23]]


***Naive Bayes Model ***


In [602]:
from sklearn.naive_bayes import GaussianNB
classifier=GaussianNB()
classifier.fit(X_train,y_train)

GaussianNB(priors=None)

In [603]:
#Prediction
y_pred=classifier.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

         no       0.90      0.98      0.94       649
        yes       0.70      0.25      0.37        92

avg / total       0.88      0.89      0.87       741



In [604]:
accuracy_score(y_test, predictions)
print(confusion_matrix(y_test,predictions))

[[639  10]
 [ 69  23]]


**SVM Model**

In [605]:
from sklearn.svm import SVC
svc_model = SVC()
svc_model.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

**Model Evaluation**

In [606]:
predictions = svc_model.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

         no       0.90      0.98      0.94       649
        yes       0.67      0.22      0.33        92

avg / total       0.87      0.89      0.86       741

